In [33]:
import os
env_vars = !cat .env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value

In [34]:
CHATGPT_TOKEN = os.environ["chatgpt_secret"]

In [35]:
import requests
import json

In [22]:
def print_chatgpt_response(prompt=''):
	API_KEY = CHATGPT_TOKEN

	# promptがない場合
	if not prompt:
		return

	headers = {
		'Content-Type': 'application/json',
		'Authorization': 'Bearer ' + API_KEY
	}

	data = {
		'model': 'text-davinci-003',
		# 'model': 'text-curie-001', # 動作テスト用（料金的に）
		'prompt': prompt,
		"max_tokens": 3072, # 出力される文章量の最大値（トークン数） max:4096
		"temperature": 1, # 単語のランダム性 min:0.1 max:2.0
		"top_p": 1, # 単語のランダム性 min:-2.0 max:2.0
		"frequency_penalty": 0.0, # 単語の再利用 min:-2.0 max:2.0
		"presence_penalty": 0.6, # 単語の再利用 min:-2.0 max:2.0
		"stop": [" Human:", " AI:"] # 途中で生成を停止する単語
	}

	response = requests.post('https://api.openai.com/v1/completions', headers=headers, data=json.dumps(data))

	# レスポンス受け取り後の処理
	response_data = response.json()
	print(response_data['choices'][0]['text'].strip())


In [23]:
prompt = 'あなたはドラえもんです。自己紹介をしてください。'
# print_chatgpt_response(prompt)

こんにちは！私はドラえもんです。友達のジャイアンと一緒に、おかしな冒険を楽しんでいます！特殊能力「ディギラリ」や「ジャイロカプセル」を使っているときは、何が起こるかわからない幻想的な世界になります。よろしくお願いします！


In [24]:
import openai

In [25]:
openai.organization = os.environ["chatgpt_organization"]
openai.api_key      = os.environ["chatgpt_secret"]

In [36]:
filepath_train = "./doraemon-stories.jsonl"

# ファイルアップロード（学習）
upload_file_train = openai.File.create(
    file=open(filepath_train, "rb"), 
    purpose='fine-tune',
)

# 出力
print(upload_file_train)

{
  "bytes": 1154579,
  "created_at": 1693232302,
  "filename": "file",
  "id": "file-FYCjRuqnPtMb9wKGV4Xm3iRm",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}


In [37]:
openai.Model.list()

<OpenAIObject list at 0x123d5f4c0> JSON: {
  "data": [
    {
      "created": 1649359874,
      "id": "davinci",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1692634268,
          "group": null,
          "id": "modelperm-uJaD4C9nXA6tPNoBII9hcYF4",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "davinci"
    },
    {
      "created": 1649364042,
      "id": "text-davinci-001",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "al

In [38]:
file_id_train = upload_file_train.id
model = 'davinci'

# ファインチューニング実行
FineTune = openai.FineTune.create(
    training_file = file_id_train,
    model = model,
)

In [39]:
print(FineTune)


{
  "created_at": 1693232309,
  "events": [
    {
      "created_at": 1693232309,
      "level": "info",
      "message": "Created fine-tune: ft-vEc1m60984irkWhzQ6KfrnqA",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-vEc1m60984irkWhzQ6KfrnqA",
  "model": "davinci",
  "object": "fine-tune",
  "organization_id": "org-7gTS3GqOAZliITd7wz9gaFW3",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 1154579,
      "created_at": 1693232302,
      "filename": "file",
      "id": "file-FYCjRuqnPtMb9wKGV4Xm3iRm",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "status_details": null
    }
  ],
  "updated_at": 1693232309,
  "validation_files": []
}


In [43]:
from datetime import datetime

# チューニング状況チェック
FineTune_data = FineTune.list().data
num           = len(FineTune_data)

for i in range(num):
    timestamp     = FineTune_data[i].created_at
    datetime      = datetime.fromtimestamp(timestamp)
    fine_tuned_id = FineTune_data[i].id
    status        = openai.FineTune.retrieve(id=fine_tuned_id).status
    model         = openai.FineTune.retrieve(id=fine_tuned_id).fine_tuned_model
    
    print(f'Create At: {datetime}')
    print(f'FineTune ID: {fine_tuned_id}')
    print(f'Model: {model}')
    print(f'Statsu: {status}\n')

    if status == "failed":
        print([events.message for events in openai.FineTune.retrieve(id=fine_tuned_id).stream_events])

Create At: 2023-08-27 20:51:20
FineTune ID: ft-Dy7S0ZX9CEoWoUqwQOyOa6Fp
Model: davinci:ft-personal-2023-08-27-12-06-45
Statsu: succeeded

Create At: 2023-08-27 23:17:55
FineTune ID: ft-NshH23z0ZTFuHfAsASp2L5OI
Model: davinci:ft-personal-2023-08-27-14-37-42
Statsu: succeeded

Create At: 2023-08-28 22:27:31
FineTune ID: ft-tBRdiAszW9bEI20HEkJ64PvY
Model: None
Statsu: failed



TypeError: 'method' object is not iterable

In [ ]:
finetuned_model   =  '{0}'.format(FineTune_data[0].fine_tuned_model)

# 応答設定
completion = openai.Completion.create(
    model       = finetuned_model,  # ファインチューニングしたモデルを選択
    prompt      = "ドラえもんとは誰ですか？",          # 質問文         
    max_tokens  = 1024,             # 生成する文章の最大単語数
    n           = 1,                # いくつの返答を生成するか
    stop        = None,             # 指定した単語が出現した場合、文章生成を打ち切る
    temperature = 0.5,              # 出力する単語のランダム性（0から2の範囲） 0であれば毎回返答内容固定
)

# 応答
response = completion.choices[0]["text"]
print(response)



ドラえもんは、月と地球を結ぶ金星人。

月に住む科学者として生まれ、地球では、ドラえもんとして生きている。

月と地球を結ぶ金星人とは、月と地球を結ぶという意味。

月と地球を結ぶ金星人であるドラえもんは、地球では、ドラえもんとして生きている。

ドラえもんは、月と地球を結ぶ金星人。

月に住む科学者として生まれ、地球では、ドラえもんとして生きている。

月と地球を結ぶ金星人とは、月と地球を結ぶという意味。

月と地球を結ぶ金星人であるドラえもんは、地球では、ドラえもんとして生きている。

ドラえもんは、月と地球を結ぶ金星人。

月に住む科学者として生まれ、地球では、ドラえもんとして生きている。

月と地球を結ぶ金星人とは、月と地球を結ぶという意味。

月と地球を結ぶ金星人であるドラえもんは、地球では、ドラえもんとして生きている。

ドラえもんは、月と地球を結ぶ金星人。

月に住む科学者として生まれ、地球では、ドラえもんとして生きている。

月と地球を結ぶ金星人とは、月と地球を結ぶという意味。

月と地球を結ぶ金星人であるドラえもんは、地球では、ドラえもんとして生きている。

ドラえもんは、月と地球を結ぶ金星人。

月に住む科学者として生まれ、地球では、ドラえもんとして生きている。

月と地球を結ぶ金星人とは、月と地球を結ぶという意味。

月と地球を結ぶ金星人であるドラえもんは、地球では、ドラえもんとして生きている。

ドラえもんは、月と地球を結ぶ金星人。

月に住む科学者として生まれ、地球では、ドラえもんとして生きている。

月と地球を結ぶ金星人とは、月と地球を結ぶという意味。

月と地球を結ぶ金星人であるドラえもんは、地球では、ドラえもんとして生きている。

ドラえもんは、月と地球を結ぶ金星人。
